## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham". 

### Данные

Для удобства данные приложены в описании задания

In [1]:
import pandas as pd
df = pd.read_csv('https://courses.openedu.ru/assets/courseware/v1/008dcf9ca6aab621d3b3831db6815867/asset-v1:ITMOUniversity+ADVMLAUTXT+spring_2023_ITMO_mag+type@asset+block/3_data.csv', encoding='latin-1')

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Оставляем только интересующие нас колонки — тексты смс и метки:

In [3]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [4]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [5]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Выполните предварительную обработку текста (как в примере).

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [7]:
import nltk

from nltk import stem
from nltk.corpus import stopwords
import re

nltk.download("stopwords")
stopwords = set(stopwords.words('english'))

stemmer = stem.SnowballStemmer('english', ignore_stopwords=True)

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text).lower()
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stopwords])
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверка, что функция работает верно

In [9]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [10]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

Разделите датасет на тренировочный и тестовый набор данных с параметрами: `test_size=0.25, random_state = 1`

In [11]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=1)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Обучите классификатор SVM при `random_state=1, C=1.1` на обучающей выборке и произведите оценку полученной модели на тестовой.

In [14]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# обучаем подель SVM
model = LinearSVC(random_state=1, C=1.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Оцените модель на тестовых данных.

Введите значение:
1. Precision (macro avg)
2. Recall (macro avg)
3. F-score (macro avg)

In [15]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.977     0.996     0.986      1128
           1      0.965     0.836     0.896       165

    accuracy                          0.975      1293
   macro avg      0.971     0.916     0.941      1293
weighted avg      0.975     0.975     0.974      1293



Выполните предсказания для следующих сообщений:
* `Poorly written I'm giving up on this book.`
* `Screw this self-help industry crap.`
* `Hello, I have a special offer for you… WANT TO LOSE WEIGHT? The most powerful weightloss is now available without prescription. All natural Adipren720 100% Money Back Guarantee!`
* `So the summer has flown by - today a new school year begins. Ahead - new students and pupils; knowledge, discoveries and accomplishments, checking abstracts and coursework, conducting colloquiums and exams.`



In [18]:
txt_1 = "Poorly written I'm giving up on this book."
txt_2 = "Screw this self-help industry crap."
txt_3 = "Hello, I have a special offer for you… WANT TO LOSE WEIGHT? The most powerful weightloss is now available without prescription. All natural Adipren720 100% Money Back Guarantee!"
txt_4 = "So the summer has flown by - today a new school year begins. Ahead - new students and pupils; knowledge, discoveries and accomplishments, checking abstracts and coursework, conducting colloquiums and exams."

txt_list = [txt_1, txt_2, txt_3, txt_4]

predictions = [model.predict(vectorizer.transform([preprocess(txt)])) for txt in txt_list]
predictions

[array([0]), array([0]), array([0]), array([0])]